In [1]:
from sklearn.decomposition import PCA
from model.TransE import *
from model.ConvKB import ConvKB
from model.utils import *
from model.Triplet import Triplet
from model.CustomTripletMarginLoss import CustomTripletMarginLoss
from argparse import Namespace
import torch
import torch.nn as nn
import sys
from torch.autograd import Variable
import torch.optim as optim
from numpy import linalg as LA
import numpy as np
from math import log10,floor
import json
import pickle,os
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import csv
from model.CapsuleNet import *

In [2]:
NUMBER_NEIGHBORS = 15
WEIGHT_PATH = "/storage/quandh/OsGenePrio_capse/data/capse_3_param2/param2/CapsE.pkl"
DEVICE = 2 

In [3]:
#Load data to show attributes of gene
with open("./support/uniprot.pkl","rb") as f:
    uniprot = pickle.load(f)
    f.close()
with open("./support/pyrice.pkl","rb") as f:
    pyrice = pickle.load(f)
    f.close()
with open("./support/id_dict_2018", "rb") as f:
    id_dict_2018 = pickle.load(f)
    f.close()
with open("./support/loc_dict_2018", "rb") as f:
    loc_dict_2018 = pickle.load(f)
    f.close()
entity_total = get_total("./data/GENE/entity2id.txt")
relation_total = get_total("./data/GENE/relation2id.txt")
processed_entity_2_id = load_data("./data/GENE/entity2id.txt", ignore_first=True)
relation_2_id = load_data("./data/GENE/relation2id.txt", ignore_first=True)
triple_list = []
processed_id_2_entity = dict()
i = 0
with open("./data/GENE/entity2id.txt", 'r') as f:
    for line in f:
        if True and i == 0:
            i += 1
            continue
        line = line.strip()
        if line == '':
            continue
        parts = line.split("\t")
        processed_id_2_entity[int(parts[1])] = parts[0]
i = 0
triple_list =[]
with open("./data/GENE/triplet2id.txt", 'r') as f:
    for line in f:
        if True and i == 0:
            i += 1
            continue
        line = line.strip()
        if line == '':
            continue
        parts = line.split("\t")
        head = int(parts[0])
        tail = int(parts[1])
        rel = int(parts[2])
        triple_list.append((head,tail,rel))

In [4]:
net = CapsuleNetwork(entity_total = entity_total, relation_total = relation_total, embedding_size =150, vec_len_secondCaps = 10, 
                     num_units = 1, num_iterations = 3, num_filters = 150)
if torch.cuda.is_available():
    device = torch.device("cuda:"+str(DEVICE) if torch.cuda.is_available() else "cpu")
    checkpoint = torch.load("/storage/quandh/OsGenePrio_capse/data/capse_3_param2/param2/CapsE.pkl", map_location = device)
    net.to(device)
else:
    checkpoint = torch.load("/storage/quandh/OsGenePrio_capse/data/capse_3_param2/param2/CapsE.pkl", map_location=lambda storage, loc: storage)
net.load_state_dict(checkpoint)
net.eval()
embedding_items = list(checkpoint.items())
# 1: entity
# 2: relation
data_train = embedding_items[0][1].cpu().numpy()

if os.path.exists("./data/GENE/capse/kNN.pkl"):
    with open("./data/GENE/capse/kNN.pkl", "rb") as f:
        nbrs = pickle.load(f)
        f.close()
    with open("./data/GENE/capse/indices.pkl", "rb") as f:
        indices = pickle.load(f)
        f.close()
    with open("./data/GENE/capse/distance.pkl", "rb") as f:
        distances = pickle.load(f)
        f.close()
else:
    nbrs = NearestNeighbors(n_neighbors=NUMBER_NEIGHBORS, algorithm='ball_tree').fit(data_train)
    distances, indices = nbrs.kneighbors(data_train)
    with open("./data/GENE/capse/kNN.pkl", "wb") as f:
        pickle.dump(nbrs,f)
        f.close()
    with open("./data/GENE/capse/indices.pkl", "wb") as f:
        pickle.dump(indices, f)
        f.close()
    with open("./data/GENE/capse/distance.pkl", "wb") as f:
        pickle.dump(distances, f)
        f.close()

In [5]:
#TEST KNN WITH GENE
while True:
    gene_name = input("\nType genename: ").strip()
    if gene_name == "":
        break
    if gene_name not in processed_entity_2_id.keys():
        print("Gene not found")
        continue
    if gene_name in processed_entity_2_id.keys():
#         print(id_dict_2018[gene_name])
#         print(gene_name)
        if gene_name in uniprot.keys():
            print("Gene UniProt: {}".format(uniprot[gene_name]))
        if gene_name in pyrice.keys():
            print("Gene PyRice: {}\n".format(pyrice[gene_name]))
    else:
        print("Att {}\n".format(gene_name))
    print("Top 5 relation: ")
    count = 0
    for index in indices[processed_entity_2_id[gene_name]][1:]:
        similar_gene = processed_id_2_entity[index]
        if similar_gene in id_dict_2018.keys():
#             print(id_dict_2018[similar_gene])
            print(similar_gene)
            if similar_gene in uniprot.keys():
                print("Gene UniProt: {}".format(uniprot[similar_gene]))
            if similar_gene in pyrice.keys():
                print("Gene PyRice:{}\n".format(pyrice[similar_gene]))
            count+=1
            if count == 15: 
                break
                print("----------------------------------------------------")
#         else:
#             print("Att {}\n".format(similar_gene))
#             count += 1
#             if count == 5: break  


Type genename: Os01g0588900
Gene UniProt: {'Protein': {"cytokinin riboside 5'-monophosphate phosphoribohydrolase log"}, 'GO': {'GO:0009691', 'GO:0005829', 'GO:0048509', 'GO:0016799', 'GO:0005634', 'GO:0016021'}, 'Keyword': {'KW-0963', 'KW-1133', 'KW-1185', 'KW-0812', 'KW-0203', 'KW-0378', 'KW-0472', 'KW-0181'}}
Gene PyRice: {'GO': {'GO:0009691', 'GO:0005829', 'GO:0048509', 'GO:0016799', 'GO:0005634'}, 'Position': {'chr01'}, 'CGSNL Gene Name': {'lonely guy 1'}, 'Trait Class': set(), 'TO': {'TO:0000017', 'TO:0002660', 'TO:0000050', 'TO:0002659', 'TO:0000089'}, 'PO': {'PO:0000230', 'PO:0000229', 'PO:0009105'}, 'Keyword': {'meristem', 'shoot', 'cytokinin'}, 'Name': set(), 'Gene name synonym(s)': {"lonely guy, lonely guy, lonely guy, protein lonely guy, cytokinin riboside 5'-monophosphate phosphoribohydrolase log"}}

Top 5 relation: 
Os06g0226500
Gene UniProt: {'Protein': {'transcription factor ili4'}, 'GO': {'GO:0009741', 'GO:0006355', 'GO:0005737', 'GO:0046983', 'GO:0005634', 'GO:0009742


Type genename: 


In [6]:
#TEST PREDICT TAIL WITH GENE
candidates = []
for att in processed_entity_2_id.keys():
    if att not in id_dict_2018.keys():
        candidates.append(att)
        
gene_id = "Os08g0490800"
#gene_id = "Os05g0180600"
relation = "GO"
sub_candidates = []
count = 0 
for att in candidates:
    if (att.find('GO:') != -1): 
        sub_candidates.append(att)
        count +=1
#     if count == 2000:
#         break
scores = dict()
h_batch = []
t_batch = []
r_batch = []
new_att = []
for att in sub_candidates:
    if ((processed_entity_2_id[gene_id],processed_entity_2_id[att],relation_2_id[relation])) in triple_list:
        #print(gen_id,att,relation)
        continue
    new_att.append(att)
    h_batch.append(processed_entity_2_id[gene_id])
    t_batch.append(processed_entity_2_id[att])
    r_batch.append(relation_2_id[relation])
print("Number items predicting: ",len(t_batch))
h_batch, t_batch, r_batch = torch.LongTensor(h_batch), torch.LongTensor(t_batch), torch.LongTensor(r_batch)
if torch.cuda.is_available():
    h_batch, t_batch, r_batch = h_batch.to(device), t_batch.to(device), r_batch.to(device)
h_batch, t_batch, r_batch = Variable(h_batch), Variable(t_batch), Variable(r_batch)
outputs, _, _, _ = net(h_batch, t_batch, r_batch)
outputs = outputs.data.tolist()
for i in range(len(outputs)):
    scores.setdefault(new_att[i],outputs[i])
sorted_x = sorted(scores.items(), key=lambda kv: kv[1])

Number items predicting:  3753


In [7]:
sorted_x

[('GO:0006351', [0.09887958317995071]),
 ('GO:0006325', [0.1193348839879036]),
 ('GO:0000790', [0.257596492767334]),
 ('GO:0046983', [2.364748477935791]),
 ('GO:0043565', [2.386559009552002]),
 ('GO:0003700', [2.4980483055114746]),
 ('GO:0006355', [2.5869486331939697]),
 ('GO:0006334', [2.7701797485351562]),
 ('GO:0045893', [2.820744037628174]),
 ('GO:0042393', [2.976146936416626]),
 ('GO:0003682', [3.044382095336914]),
 ('GO:0044212', [3.052321195602417]),
 ('GO:0001228', [3.0633270740509033]),
 ('GO:0005730', [3.1277081966400146]),
 ('GO:0051726', [3.13449764251709]),
 ('GO:0003690', [3.1389667987823486]),
 ('GO:0000122', [3.1420812606811523]),
 ('GO:0000785', [3.1423263549804688]),
 ('GO:0030154', [3.143362283706665]),
 ('GO:0008270', [3.1446409225463867]),
 ('GO:0003712', [3.1449406147003174]),
 ('GO:0008134', [3.147341728210449]),
 ('GO:0003714', [3.147581100463867]),
 ('GO:0045944', [3.14762282371521]),
 ('GO:0000981', [3.1476995944976807]),
 ('GO:0007275', [3.147831439971924]),
